In [1]:
# no standard scaling
# with generated quantities
# bad Rhat

In [2]:
import numpy as np
from numpy.random import *
import pystan
import pandas as pd
import matplotlib.pyplot as plt
import arviz as az
from scipy.special import expit

In [3]:
# 「Pythonによる因果分析」第４章のデータを使った

In [4]:
num_data = 500

x_1 = randint(15, 76, num_data).astype('float')
#e_z = randn(num_data)
e_z = 0

z_base = x_1 - 40 + 1*e_z
z_prob = expit(z_base)

Z = np.array([])
for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0]
    Z = np.append(Z, Z_i)

#e_y = randn(num_data)
e_y = 0 

Y = -x_1 + 40*Z + 100 + 2*e_y

In [5]:
df = pd.DataFrame({'Age': x_1,
                   'CM': Z,
                   'Sale_amount': Y,
                   })

df.head(10)

,Age,CM,Sale_amount
0,68.0,1.0,72.0
1,63.0,1.0,77.0
2,38.0,1.0,102.0
3,73.0,1.0,67.0
4,37.0,0.0,63.0
5,35.0,0.0,65.0
6,73.0,1.0,67.0
7,51.0,1.0,89.0
8,25.0,0.0,75.0
9,70.0,1.0,70.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          500 non-null    float64
 1   CM           500 non-null    float64
 2   Sale_amount  500 non-null    float64
dtypes: float64(3)
memory usage: 11.8 KB


In [7]:
Sale_amount = df['Sale_amount']
Age = df['Age']
CM = df['CM']

stan_data = {
    'N': num_data,
    'Sale_amount': Sale_amount,
    'Age': Age,
    'CM': CM
}

In [8]:
stan_code = """
data {
    int N;
    real Sale_amount[N];
    vector[N] Age;
    vector[N] CM;
}

parameters {
    real<upper=0> Intercept;
    real b_Age;
    real<lower=0> sigma;
    real a;
    real b;
    real c;
}

model {
    vector[N] prob = inv_logit(Intercept + b_Age * Age);
    Sale_amount ~ normal(a + b * Age + c * prob, sigma);
}

generated quantities {
    vector[N] Sales_amount_pred;
    for (i in 1:N) {
        Sales_amount_pred[i] = a + b * Age[i] + c * inv_logit(Intercept + b_Age * Age[i]);
    }
}
"""

In [9]:
sm = pystan.StanModel(model_code= stan_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_7c1b1bb4a76c136b3b65a9faa2479a8d NOW.


In [10]:
mcmc_result = sm.sampling(
    data=stan_data,
    chains=3,
    iter=3000,
    warmup=2000,
    thin=1
)

In [11]:
print(mcmc_result)

Inference for Stan model: anon_model_7c1b1bb4a76c136b3b65a9faa2479a8d.
3 chains, each with iter=3000; warmup=2000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=3000.

                         mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
Intercept              -1.0e4  795.04 1201.5 -1.3e4 -1.1e4  -9837  -9240  -8413      2   2.35
b_Age                  253.75   19.88  30.04 210.34 231.01 245.93 275.33 318.74      2   2.35
sigma                    5.71    0.05   0.18   5.39   5.59   5.69   5.82   6.11     15    1.2
a                       98.31    0.21   0.81  96.99  97.73  98.22  98.81 100.13     15   1.19
b                       -0.89  6.9e-3   0.03  -0.95  -0.91  -0.89  -0.87  -0.85     13   1.27
c                       34.97    0.23   0.84  33.46  34.42  34.87  35.52  36.88     14   1.32
Sales_amount_pred[1]     72.7    0.05   0.42  71.84  72.41  72.71  72.99  73.53     66   1.03
Sales_amount_pred[2]    77.15    0.02   0.36  76.43  76.92 

In [12]:
Sales_amount_pred = mcmc_result['Sales_amount_pred'].mean(axis = 0)
df['Sales_amount_pred'] = Sales_amount_pred
pd.set_option('display.max_rows', None)
df

,Age,CM,Sale_amount,Sales_amount_pred
0,68.0,1.0,72.0,72.699696
1,63.0,1.0,77.0,77.154035
2,38.0,1.0,102.0,64.452679
3,73.0,1.0,67.0,68.245356
4,37.0,0.0,63.0,65.343547
5,35.0,0.0,65.0,67.125283
6,73.0,1.0,67.0,68.245356
7,51.0,1.0,89.0,87.844451
8,25.0,0.0,75.0,76.033962
9,70.0,1.0,70.0,70.917960
